# Canny Benchmark

## Imports & Setup

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append("../")

if False:
    # Enable the CUDA simulator
    os.environ["NUMBA_OPT"] = "0"
    os.environ["NUMBA_ENABLE_CUDASIM"] = "1"
    os.environ["NUMBA_CUDA_DEBUGINFO"] = "1"

import logging
import time
import pathlib
from pathlib import Path
import importlib.util
from importlib import reload
import math
from io import BytesIO
import unittest.mock as mock
import gc

import matplotlib as mpl
import matplotlib.pyplot as plt

from icecream import ic

import numpy as np
import pandas as pd
import scipy as sp
import numba
from numba import cuda

import cv2

from IPython.display import clear_output, display, Image
import ipywidgets as widgets
from PIL import Image as PILImage

from utils.benchmarking import time_function, time_line
from utils.setup_notebook import init_notebook, source_code_path_is_from_notebook
from utils.setup_logging import setup_logging
from utils.plotting_tools import (
    SmartFigure,
    to_ipy_image,
    plot_image,
    plot_kernel,
    plot_matrix,
)

init_notebook()
setup_logging("INFO")


## Loading Canny Implementations

In [2]:
# Load canny implementations
dir_canny_impls = "./canny_impls"
files_canny_impls = [f for f in os.listdir(dir_canny_impls) if str(f).endswith(".py")]
files_canny_impls.sort()

if "canny_impls" not in locals():
    canny_impls = []


def load_module(file_path: str, dyn_modules: list):
    module_name = Path(file_path).stem
    module_string = f"{module_name}"

    already_loaded = module_string in sys.modules
    if already_loaded:
        with time_line(f"Reloading {module_name}"):
            module = sys.modules[module_string]
            reload(module)
        return module

    with time_line(f"Loading {module_name}"):
        folder = str(Path(file_path).parent)
        if folder not in sys.path:
            sys.path.append(folder)
        spec = importlib.util.spec_from_file_location(module_string, file_path)
        module = importlib.util.module_from_spec(spec)
        sys.modules[module_string] = module
        spec.loader.exec_module(module)

    dyn_modules.append(module)

    return module


with time_line(f"Loading {len(files_canny_impls)} canny implementations"):
    for f in files_canny_impls:
        full_path = Path(dir_canny_impls) / f
        module = load_module(full_path, canny_impls)


2024-10-22 17:50:40.483 INFO root Loading 2 canny implementations started (C:\Users\_\AppData\Local\Temp\ipykernel_24804\3814923242.py:35)
2024-10-22 17:50:40.517 INFO root Loading rd_numba_cuda_fp32 started (C:\Users\_\AppData\Local\Temp\ipykernel_24804\3814923242.py:21)
2024-10-22 17:50:40.522 INFO numba.cuda.cudadrv.driver init
2024-10-22 17:50:42.950 INFO root Loading rd_numba_cuda_fp32 took: 2.4331 s (C:\Users\_\AppData\Local\Temp\ipykernel_24804\3814923242.py:21)
2024-10-22 17:50:43.019 INFO root Loading rd_vec_v4_dibit started (C:\Users\_\AppData\Local\Temp\ipykernel_24804\3814923242.py:21)
2024-10-22 17:50:44.086 INFO root Loading rd_vec_v4_dibit took: 1.0659 s (C:\Users\_\AppData\Local\Temp\ipykernel_24804\3814923242.py:21)
2024-10-22 17:50:44.088 INFO root Loading 2 canny implementations took: 3.6038 s (C:\Users\_\AppData\Local\Temp\ipykernel_24804\3814923242.py:35)


## Loading Input Images

In [3]:
image_input_dir = "./image_input"
input_files = [
    f for f in os.listdir(image_input_dir) if f.endswith(".jpg") or f.endswith(".png")
]
input_files.sort()

images_color = []
images_gray = []

with time_line(f"Loading {len(input_files)} images"):
    for f in input_files:
        with time_line(f"Loading {f}"):
            file_image = Path(image_input_dir) / f
            image_color = cv2.imread(str(file_image), cv2.IMREAD_COLOR)
            images_color.append(image_color)
            # image_gray = (
            #    cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
            # )
            image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
            images_gray.append(image_gray)

# sort all three lists by image size
images_color, images_gray, input_files = zip(
    *sorted(zip(images_color, images_gray, input_files), key=lambda x: x[0].size)
)

2024-10-22 17:50:44.597 INFO root Loading 5 images started (C:\Users\_\AppData\Local\Temp\ipykernel_24804\2264552153.py:10)
2024-10-22 17:50:44.632 INFO root Loading circle_128.png started (C:\Users\_\AppData\Local\Temp\ipykernel_24804\2264552153.py:12)
2024-10-22 17:50:44.635 INFO root Loading circle_128.png took: 1.6396 ms (C:\Users\_\AppData\Local\Temp\ipykernel_24804\2264552153.py:12)
2024-10-22 17:50:44.666 INFO root Loading circle_32.png started (C:\Users\_\AppData\Local\Temp\ipykernel_24804\2264552153.py:12)
2024-10-22 17:50:44.668 INFO root Loading circle_32.png took: 698.6000 µs (C:\Users\_\AppData\Local\Temp\ipykernel_24804\2264552153.py:12)
2024-10-22 17:50:44.697 INFO root Loading circle_64.png started (C:\Users\_\AppData\Local\Temp\ipykernel_24804\2264552153.py:12)
2024-10-22 17:50:44.699 INFO root Loading circle_64.png took: 644.3000 µs (C:\Users\_\AppData\Local\Temp\ipykernel_24804\2264552153.py:12)
2024-10-22 17:50:44.723 INFO root Loading park.png started (C:\Users\_\A

## Running Canny

In [14]:
# Show a dropdown to select the image

output = widgets.Output()

image_dropdown = widgets.Dropdown(
    options=input_files,
    description="Image",
    disabled=False,
)
canny_impl_dropdown = widgets.Dropdown(
    options=[canny_impl.__name__ for canny_impl in canny_impls],
    description="Canny Implementation",
    disabled=False,
    value=canny_impls[0].__name__,
)
sigma_slider = widgets.FloatSlider(
    value=3.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description="Sigma",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".1f",
)

fig_width = 16
image_size = 512

plot_gauss_smart_fig = SmartFigure()
plot_non_max_smart_fig = SmartFigure()


@output.capture(clear_output=True, wait=True)
def on_menu_change(change=None):
    # reload the impl module
    current_canny_impl = canny_impl_dropdown.value
    load_module(Path(dir_canny_impls) / f"{current_canny_impl}.py", canny_impls)

    # Show the selected image
    image_color = images_color[image_dropdown.index]
    image_gray = images_gray[image_dropdown.index]

    height, width = image_gray.shape

    logging.info(f"Selected image: {image_dropdown.value} ({width}x{height})")
    logging.info(f"Selected canny implementation: {canny_impl_dropdown.value}")
    with time_line("Displaying input images"):
        display(
            widgets.HBox(
                [
                    to_ipy_image(image_color, longest_side=image_size, upscale=True),
                    to_ipy_image(image_gray, longest_side=image_size, upscale=True),
                ]
            )
        )

    canny_impl = next(
        canny_impl
        for canny_impl in canny_impls
        if canny_impl.__name__ == canny_impl_dropdown.value
    )

    # GAUSS
    with time_line("Blurring image"):
        image_blurred = canny_impl.blur_gauss(image_gray, sigma_slider.value)
    with time_line("Displaying blurred image"):
        display(to_ipy_image(image_blurred, longest_side=image_size, upscale=True))

    plot_gauss = False
    if plot_gauss:
        global plot_gauss_smart_fig
        plot_gauss_smart_fig = SmartFigure(
            figsize=(
                fig_width,
                fig_width / 2,
            ),  # rc_params={"figure.constrained_layout.use": True}
        )
        fig_gauss = plot_gauss_smart_fig.get_fig()
        ax = fig_gauss.add_subplot(1, 2, 1), fig_gauss.add_subplot(1, 2, 2)
        for ax_ in ax:
            if hasattr(ax_, "clear"):
                ax_.clear()
        plot_matrix(ax[0], image_gray, title="Original Image")
        plot_matrix(ax[1], image_blurred, title="Blurred Image")

        # ax_3d = fig.add_subplot(2, 2, 3, projection="3d")
        # kernel = image_blurred[:19, :19]
        # plot_kernel(ax_3d, kernel, title="Kernel")

        fig_gauss.tight_layout()
        fig_gauss.canvas.layout.min_width = "400px"
        fig_gauss.canvas.layout.flex = "1 1 auto"
        fig_gauss.canvas.layout.width = "auto"
        # fig.canvas.resizable = False
        display(fig_gauss.canvas)

    # SOBEL
    with time_line("Calculating sobel gradients"):
        grad_mag, grad_dir = canny_impl.sobel_gradients(image_blurred)

    grad_mag = grad_mag.astype(np.float32)
    grad_dir = grad_dir.astype(np.float32)

    grad_dir_color = cv2.applyColorMap(
        np.uint8((grad_dir + np.pi) / (2 * np.pi) * 255),
        cv2.COLORMAP_RAINBOW,
    )
    grad_dir_color = grad_dir_color.astype(np.float32) / 255.0
    image_gradients = np.append(
        cv2.cvtColor(grad_mag, cv2.COLOR_GRAY2BGR), grad_dir_color, axis=1
    )
    display(to_ipy_image(image_gradients, longest_side=image_size, upscale=True))

    # NON-MAXIMUM SUPPRESSION
    with time_line("Non-maximum suppression"):
        image_nms = canny_impl.non_max(grad_mag, grad_dir)
    with time_line("Displaying non-maximum suppression"):
        display(to_ipy_image(image_nms, longest_side=image_size, upscale=True))

    plot_non_max = True
    if plot_non_max:
        global plot_non_max_smart_fig
        plot_non_max_smart_fig = SmartFigure(
            figsize=(
                fig_width,
                fig_width / 2,
            ),  # rc_params={"figure.constrained_layout.use": True}
        )
        fig_non_max = plot_non_max_smart_fig.get_fig()
        ax = fig_non_max.add_subplot(1, 2, 1), fig_non_max.add_subplot(1, 2, 2)
        for ax_ in ax:
            if hasattr(ax_, "clear"):
                ax_.clear()
        plot_matrix(ax[0], grad_mag, title="Gradient Magnitude")
        plot_matrix(ax[1], image_nms, title="Non-Maximum Suppression")

        fig_non_max.tight_layout()
        fig_non_max.canvas.layout.min_width = "400px"
        fig_non_max.canvas.layout.flex = "1 1 auto"
        fig_non_max.canvas.layout.width = "auto"
        # fig.canvas.resizable = False
        display(fig_non_max.canvas)


image_dropdown.observe(on_menu_change, names="value")
canny_impl_dropdown.observe(on_menu_change, names="value")
sigma_slider.observe(on_menu_change, names="value")

display(
    widgets.VBox(
        [widgets.HBox([image_dropdown, canny_impl_dropdown, sigma_slider]), output]
    )
)
on_menu_change()